In [5]:
import itertools
import multiprocessing
pool = multiprocessing.Pool(1)  #allows to use multiple cores
import weight_decider #a module containing some of the necessary functions, especially the ones that use multiprocessing
import time
import numpy as np
import os
#путин-хуйло


def flatten(list_of_lists):
    if len(list_of_lists) == 0:
        return list_of_lists
    if isinstance(list_of_lists[0], list):
        return flatten(list(list_of_lists)[0]) + flatten(list(list_of_lists)[1:])
    return list_of_lists[:1] + flatten(list_of_lists[1:])

#filter that can use multiprocessing
def pool_filter(pool,func,candidates):
    return [c for c, keep in zip(candidates,pool.starmap(func,candidates)) if keep]

#generates all possible tuples of length l with entries from 0 to N
def lN(l,N):
    M=range(0,N+1)
    return itertools.product(M, repeat=l)



#INPUT: matrix M (list of lists)
#OUTPUT: list of indices of the columns of M sorted in the following way:
#we take the top entry of the column, then add 1/100 of the next entry, 1/(100)^2 and so on...we put the smallest such number in front.  
#works for values of the entries of M smaller than 100
def sort_columns(M):
    ind_list=[i for i in range(0,len(M[0]))]
    def f1(i):
        out=0
        for k in range(0,len(M)):
            out=out+(1/100)**k*M[k][i]
        return out
    M2=sorted(ind_list, key=f1)
    return M2


#INPUT: matrix M
#OUTPUT: matrix M with columns rearranged with respect to the ordering from sort_columns()
def sort_rel(M):
    sc=sort_columns(M)
    L=matrix(M).columns()
    out=[0 for j in range(0,len(L))]
    for k in range(0,len(L)):
        out[k]=L[sc[k]]
    return tuple(tuple(x) for x in matrix(out).columns())


#INPUT: set of words of a linear code C
#OUTPUT: all possible permutatons of indices in the words of C, i.e. permutations of columns of the generating matrix
def permutations_code(C):
    d=len(list(C)[0])
    L=[i for i in range(0,d)]
    perms=itertools.permutations(L)
    out=[]
    for x in perms:
        Cperm=[]
        for y in C:
            Cperm+=[tuple([y[j] for j in x])]
        out=out+[Cperm]
    return out


#INPUT: two generating matrices G1 and G2 over ZZ_N
#OUTPUT: True if G1 and G2 generate equivalent linear codes
def codes_equivalence(G1,G2,N):
    d1=len(G1)
    d2=len(G2)
    coeffs1=matrix(list(lN(d1,N-1)))
    coeffs2=matrix(list(lN(d2,N-1)))
    words1=list(set((coeffs1*matrix(G1)%N).rows()))
    words2=set([tuple(y) for y in (coeffs2*matrix(G2)%N).rows()])
    perm=permutations_code(words1)
    for x in perm:
        if set(x)==set(words2):
            return True
            break
    return False


#INPUT: a list of numbers x and two natural numbers r and j
#OUTPUT: returns True if x[j] is not zero and all the x[0],...,x[r-1] are 0 except for x[j]
def zeroes_positions(x,r,j):
    if x[j]==0:
        return False
    decider=1
    for k in range(0,r):
        if k!=j:
            if x[k]==0:
                decider=decider*1
            else:
                decider=decider*0
    if decider==1:
        return True
    else:
        return False 


#INPUT: dimension d, natural number N that defines that the computations are mod N,
#number of rows r and natural number s such that the entries of the matricies are from 0 to s
#OUTPUT: set of generating matrices with r rows such that any restriction to r-1 rows contains a column of zeroes
def degenerate_restrictions(pool,d,N,r,s):
    one_row00=lN(d+1,s)
    one_row=list(filter(lambda x: (x.count(0)>=r-1)&(x.count(0)<d),one_row00))
    multiargs=[(x,N) for x in one_row]
    one_row=pool_filter(pool,weight_decider.mod_N,multiargs)
    one_row=[x[0] for x in one_row] 
    
    rows={}
    
    for j in range(0,r):
        Sj=[]
        for x in one_row:
            if zeroes_positions(x,r,j)==True:
                Sj+=[x]
        rows[j]=Sj
    rows=[rows[j] for j in range(0,r)]
    rels=list(itertools.product(*rows))
    X=list(set(sort_rel(x) for x in rels))
    X=list(filter(lambda x: gcd(matrix(x).list())==1, X))
    multiargs=[[x] for x in X]
    X=[x[0] for x in pool_filter(pool,weight_decider.column_of_zeroes_check,multiargs)]
    return X


#INPUT: a set of generating matrices S and a natural number N modulo which everything is computed
#OUTPUT: the subset of S of matrices that generate a thin linear code, the permutationally equivalent matrices are moded away
def l_check(pool,S,N):
    X=S
    for eps in range(1,N):
        multiargs=[(x,N,eps) for x in X]
        X=pool_filter(pool,weight_decider.WC_eps,multiargs)
        X=[x[0] for x in X]

    multiargs=[[x,X] for x in X]
    out=pool_filter(pool,weight_decider.is_perm_in,multiargs)
    out=[x[0] for x in out]
    return out

#This function is basically identical, but permutation equivalence of matrices check is relaxed to checking the equality of weight enumerators and h^*-polynomials
def l_check_experimental(pool,S,N):
    X=S
    for eps in range(1,N):
        multiargs=[(x,N,eps) for x in X]
        X=pool_filter(pool,weight_decider.WC_eps,multiargs)
        X=[x[0] for x in X]
    multiargs=[[x,X,N] for x in X]
    out=pool_filter(pool,weight_decider.weight_ehrhart_check,multiargs)
    out=[x[0] for x in out]
    return out




#INPUT: d = the dimension, N=an integer >1, nv=maximal number of rows in the generating matrix
#OUTPUT: a list of thin simplices that are not pyramids with the given parameters and printing for each simplex a generating matrix, weight_enumerator and h*.
def thin_codes(pool,d,N,nv):
    thin={} #this will be a dictionary that given a number n would return thin simplices generated by n rows
    
    
    #creating the set of rows that sum to 0 mod N of length d+1 with at least 1 zero and no more than d-1 zeroes
    one_row00=lN(d+1,N-1)
    multiargs=[(x,d) for x in one_row00]
    one_row0=pool_filter(pool,weight_decider.zero_count,multiargs)
    multiargs=[(x[0],N) for x in one_row0]
    one_row=pool_filter(pool,weight_decider.mod_N,multiargs)
    one_row=[[x[0]] for x in one_row]
    
    
    #the one row case
    out1=[list(y) for y in set(sort_rel(x) for x in one_row)]
    thin[1]=out1
    if nv==1:
        return out1
    

    #two rows
    
    #creating possible matrices with two rows by taking a product of the set of sorted rows and unsorted
    X=list(itertools.product(out1,one_row)) 
    X=[x[0]+x[1] for x in X]
    
    #sorting the columns, filtering by gcd=1 and discarding the matrices with a column of zeroes
    X=list(set(sort_rel(x) for x in X))
    X=list(filter(lambda x: gcd(matrix(x).list())==1, X))
    multiargs=[[x] for x in X]
    X=[x[0] for x in pool_filter(pool,weight_decider.column_of_zeroes_check,multiargs)]
    
    #checking for thinness
    X=l_check(pool,X,N)
    out2=[X[0]]
    
    #discarding equivalent codes
    for x in X:
        decider=1
        for y in out2:
            if codes_equivalence(y,x,N)==True:
                decider=decider*0
            else:
                decider=decider*1
        if decider==1:
            out2+=[x]
    thin[2]=out2

    if nv==2:
        return out2

    one_row=[tuple(x) for x in one_row]
    
    
    # >2 rows
    
    
    #this subfunction returns the thin codes generated by m rows based on the thin codes generated by <m rows
    def subthin(pool,m):
        S=thin[m-1]
        X=list(itertools.product(S,one_row))
        X=[x[0]+x[1] for x in X]
        
        X=list(set(sort_rel(x) for x in X))
        X=list(filter(lambda x: gcd(matrix(x).list())==1, X))
        X=X+degenerate_restrictions(pool,d,N,m,N-1) #adding also the matrices such that all restrictions to m-1 rows give a matrix with a column of zeroes  

        multiargs=[[x] for x in X]
        X=[x[0] for x in pool_filter(pool,weight_decider.column_of_zeroes_check,multiargs)]
  
        
  
        
        #checking thinness
        X=l_check(pool,X,N)
        out=[X[0]]
        
        #discarding equivalent
        for x in X:
            decider=1
            for y in out:
                if codes_equivalence(y,x,N)==True:
                    decider=decider*0
                else:
                    decider=decider*1
            if decider==1:
                out+=[x]
                
        thin[m]=out #defining a dictionary value
        return out
    
    
    #combining the thin simplices generated by matrices with <=m rows and discarding equivalent
    for m in range(3,nv+1):
        subthin(pool,m)
    X=[]
    for i in range(2,nv+1):
        X+=thin[i]
    out=[X[0]]
    for x in X:
        decider=1
        for y in out:
            if codes_equivalence(y,x,N)==True:
                decider=decider*0
            else:
                decider=decider*1
        if decider==1:
            out+=[x]
    
    
    R=PolynomialRing(ZZ,'t')
    t=R.gen()
    R2=PolynomialRing(ZZ,'Y')
    Y=R2.gen()
    
    #This function decides if both the weight enumerator and the h^*-polynomial factorize
    #INPUT: a generating matrix x
    #OUTPUT: 'FREE JOIN likely' if both polynomials factorize, ' ' - else. 
    def FJ(x):
        h=h_star_code(x,N)
        w=WC(x,N)
        l1=list(factor(R(h)))
        f1=sum(x[1] for x in l1 )
        l2=list(factor(R2(w)))
        f2=sum(x[1] for x in l2 )
        if (f1>1)&(f2>1):
            return 'FREE JOIN likely'
        else:
            return '   '
        
        
        
    
    A=[(out.index(x),'G=',matrix(x),'h*=',h_star_code(x,N),'W=',WC(x,N),'f_h',factor(R(h_star_code(x,N))), 'f_W',factor(R2(WC(x,N))), FJ(x)) for x in out]
    
    #writting the result into a file 
    folder0=os.getcwd()
    file = open(folder0+"/ThinSimplices/ThinSimplices"+str(d)+"_"+str(N)+".txt", "a")
    file.truncate(0)
    file.write(str(out)+'\n')
    for x in out:
        file.write(str([out.index(x),N,x,h_star_code(x,N),WC(x,N),FJ(x)])+'\n')
    file.close()
    
    return out,A


#INPUT: generating matrix C over ZZ_q
#OUTPUT: h^*-polynomial of the corresponding linear code
def h_star_code(C,q):
    G=matrix(C)
    R=QQ['t']
    t=R.gen()
    k=len(C)
    codelength=len(C[0])
    coefficient_list=lN(k,q-1)
    codewords0=[]
    for coef in coefficient_list:
        codewords0=codewords0+[tuple((matrix(coef)*G % q)[0])]
    codewords=set(codewords0)
    out=0
    for c in codewords:
        h=sum(c)/q
        out=out+t^h
    return out


#INPUT: generating matrix C over ZZ_q
#OUTPUT: weight enumerator of the corresponding code
def WC(C,q):
    l=len(C[0])
    k=len(C)
    G=matrix(C)
    R=QQ['X']
    X=R.gen()
    codelength=len(C[0])
    coefficient_list=lN(k,q-1)
    codewords0=[]
    for coef in coefficient_list:
        codewords0=codewords0+[tuple((matrix(coef)*G % q)[0])]
    codewords=set(codewords0)
    W=0
    for c in codewords:
        mwt=c.count(0)
        h=X**(l-mwt)
        W=W+h
            
    return W









#this function gives a list of thin codes with parameters d,N,nv, but not all of them!
#In particular, checking permutation equivalence is costly, so we disregard codes with the same weight enumerator and h^*-polynomial
#Moreover, instead of considering matrices with entries from 0 up to N-1 we can take a smaller interval and consider the entries only up to N-1-eps
#Here we can also skip adding matrices that restrict to degenerate one by setting pyr=0 and we can still add them with pyr=1
#Doing all this we surely miss a lot of examples, but at the same time it still gives plenty of them and relatively fast
def thin_codes_experimental(pool,d,N,nv,eps,pyr):
    s=N-1-eps
    t0=time.time()
 
        
    R=PolynomialRing(ZZ,'t')
    t=R.gen()
    R2=PolynomialRing(ZZ,'Y')
    Y=R2.gen()
    
    def FJ(x):
        h=h_star_code(x,N)
        w=WC(x,N)
        l1=list(factor(R(h)))
        f1=sum(x[1] for x in l1 )
        l2=list(factor(R2(w)))
        f2=sum(x[1] for x in l2 )
        if (f1>1)&(f2>1):
            return 'FREE JOIN likely'
        else:
            return ' '
    
    
    thin={}

    
    one_row00=lN(d+1,s)
    
    multiargs=[(x,d) for x in one_row00]
    one_row0=pool_filter(pool,weight_decider.zero_count,multiargs)

    multiargs=[(x[0],N) for x in one_row0]
    one_row=pool_filter(pool,weight_decider.mod_N,multiargs)
    one_row=[[x[0]] for x in one_row]
    
    
    #one row
    out1=[list(y) for y in set(sort_rel(x) for x in one_row)]
    thin[1]=out1
    if nv==1:
        return out1
    

    #two rows
    X=list(itertools.product(out1,one_row))
    X=[x[0]+x[1] for x in X]
    
    X=list(set(sort_rel(x) for x in X))
    X=list(filter(lambda x: gcd(matrix(x).list())==1, X))
    multiargs=[[x] for x in X]
    X=[x[0] for x in pool_filter(pool,weight_decider.column_of_zeroes_check,multiargs)]
    
    t1=time.time()
    print('number of 2-dim ', len(X),t1-t0)
    
    
    print('checking weights ',2)
    X=l_check_experimental(pool,X,N)
    t2=time.time()
    print('done 2',t2-t0)


    print('equiv check 2')
    out=[X[0]]
    for x in X:
        decider=1
        for y in out: 
            if (WC(x,N)!=WC(y,N)) or (h_star_code(x,N)!=h_star_code(y,N)):
                decider=decider*1
            else:
                decider=decider*0
        if decider==1:
            out+=[x]
    thin[2]=out
    print('equiv check 2 done')
    if nv==2: 
        return out
    
    
    
    folder0=os.getcwd()
    file = open(folder0+"/ThinSimplices/experimental_ThinSimplices"+str(d)+"_"+str(N)+".txt", "a")
    file.truncate(0)
    file.write(str(out)+'\n')
    for x in out:
        file.write(str([out.index(x),N,x,h_star_code(x,N),WC(x,N),FJ(x)])+'\n')
    file.close()

    
    
    one_row=[tuple(x) for x in one_row]
    #>2 rows
    def subthin(pool,m):
        S=thin[m-1]
        X=list(itertools.product(S,one_row))
        X=[x[0]+x[1] for x in X]
        X=list(set(sort_rel(x) for x in X))
        X=list(filter(lambda x: gcd(matrix(x).list())==1, X))
        if pyr==1:
            X=X+degenerate_restrictions(pool,d,N,m,s)
        print('len ',m, 'is', len(X))
        multiargs=[[x] for x in X]
        X=[x[0] for x in pool_filter(pool,weight_decider.column_of_zeroes_check,multiargs)]
        t1=time.time()
        
        print('number of '+str(m)+'-dim ', len(X),t1-t0)
        
        
        
        print('checking weights ',m)
        X=l_check_experimental(pool,X,N)
        print('done')
        t2=time.time()
        print(t2-t0)
        
        
        print('equiv check ', m)
        out=[X[0]]
        for x in X:
            decider=1
            for y in out: 
                if (WC(x,N)!=WC(y,N)) or (h_star_code(x,N)!=h_star_code(y,N)):
                    decider=decider*1
                else:
                    decider=decider*0
            if decider==1:
                out+=[x]
        thin[m]=out
        print('equiv check done ', m)
        
        
        
        X=[]
        for i in range(2,m+1):
            X+=thin[i]
        out_file=[X[0]]
        for x in X:
            decider=1
            for y in out_file: 
                if (WC(x,N)!=WC(y,N)) or (h_star_code(x,N)!=h_star_code(y,N)):
                    decider=decider*1
                else:
                    decider=decider*0
            if decider==1:
                out_file+=[x]
        
        
        folder0=os.getcwd()
        file = open(folder0+"/ThinSimplices/experimental_ThinSimplices"+str(d)+"_"+str(N)+".txt", "a")
        file.truncate(0)
        file.write(str(out_file)+'\n')
        for x in out_file:
            file.write(str([out_file.index(x),N,x,h_star_code(x,N),WC(x,N),FJ(x)])+'\n')
        file.close()
        
        
        
        return out
    
    
    
    
    for m in range(3,nv+1):
        subthin(pool,m)
    X=[]
    for i in range(2,nv+1):
        X+=thin[i]

    out=[X[0]]
    for x in X:
        decider=1
        for y in out: 
            if (WC(x,N)!=WC(y,N)) or (h_star_code(x,N)!=h_star_code(y,N)):
                decider=decider*1
            else:
                decider=decider*0
        if decider==1:
            out+=[x]
    

        
    A=[(out.index(x),'G=',matrix(x),'h*=',h_star_code(x,N),'W=',WC(x,N),'f_h',factor(R(h_star_code(x,N))), 'f_W',factor(R2(WC(x,N))), FJ(x)) for x in out]

    folder0=os.getcwd()
    file = open(folder0+"/ThinSimplices/experimental_ThinSimplices"+str(d)+"_"+str(N)+".txt", "a")
    file.truncate(0)
    file.write(str(out)+'\n')
    for x in out:
        file.write(str([out.index(x),N,x,h_star_code(x,N),WC(x,N),FJ(x)])+'\n')
    file.close()
    
    return out,A






In [8]:
#INPUT: coordinates of the vertices of a simplex 
#OUTPUT: the list of words in the corresponding code over ZZ_N and N 
def code_of_simplex(pts):
    d=len(pts[0])
    V=factorial(d)*Polyhedron(pts).volume()
    pts1=[list(x)+[1] for x in pts]
    I=list(range(0,d+1))
    ind=list(filter(lambda x: len(x)>=2, list(subsets(I))))
    zer=[0 for i in range(0,d+1)]
    box=[zer]
    for m in ind:
        vert=sum(vector(pts1[i]) for i in m)
        box+=[vert]
    Paral=Polyhedron(box)
    paral_pts=Paral.integral_points()
    
    vert_inv=matrix(pts1).inverse()
    Lambda=[]
    for x in paral_pts:
        lambd=matrix(x)*vert_inv
        decider=1
        for y in lambd[0]:
            if (y>=0)&(y<1):
                decider*=1
            else:
                decider*=0
        if decider==1:
            Lambda+=[lambd]
    
    dens=[]
    for z in Lambda:
        for z1 in z:
            dens+=[denominator(z1)]
    N=lcm(dens)
    C=[N*l for l in Lambda]
    return set([tuple(w[0]) for w in C]),N


#returns G(q) such that \sum_{n\ge 0} P(n)q^n = G(q)/(1-q)^{d+1}, where d=deg(P)
def vtrans(p):
    q = p.parent().gen()
    d = p.degree()
    g = ((1-q)**(d+1)*sum(p(n)*q**n for n in range(d+1))).truncate(d+1)
    return g


def h_star(pts):
    P=Polyhedron(pts)
    d=P.dimension()
    return vtrans(P.ehrhart_polynomial())


#This function gives back the simplex corresponding to a given linear code
#The parameters ss and ss2 is some kind of precision parameter. Usually ss=1 and ss2=3 works fine, but it might be not enough with dimension growing.
#if the function returns a mistake - try to increase ss. 
#INPUT: G - geenrating matrix pver ZZ_N, N - N_Delta, natural numbers ss and ss2. 
#OUTPUT: coordinates of the corresponding simplex

def simplex_from_code(G,N,ss,ss2):

    
    r=len(G)
    d=len(G[0])-1
    coeffs=matrix(list(lN(r,N-1)))
    words=list(set((coeffs*matrix(G)%N).rows()))
    NN=[N for i in range(0,d+1)]
    vert=diagonal_matrix(NN).rows()
    
    
    def sparse_one(d,j):
        out=[]
        for k in range(0,d+1):
            if k!=j:
                out+=[0]
            else:
                out+=[1]
        return out
    
    
    vert_coeff_0=[sparse_one(d,i) for i in range(0,d+1)]
    
    vert_lattice_0=list(set((matrix(vert_coeff_0)*diagonal_matrix(NN)).rows()))
    vert_coeff_1=lN_sc(d+1,ss)

    vert_lattice_1=list(set((matrix(vert_coeff_1)*diagonal_matrix(NN)).rows()))
    vert_lattice=vert_lattice_1
    vert_lattice+=[tuple(-vector(x)) for x in vert_lattice]


    lattice=[tuple(vector(x[0]) + vector(x[1])) for x in list(itertools.product(words,vert_lattice))]
    lattice=list(filter(lambda x: sum(x)==N,lattice))

    M0=ZZ^(d+1)
    M=M0.submodule(lattice)
    basis=M.basis()
    
    basis_new=[]
    for x in basis:
        if sum(x)==N:
            basis_new+=[x]
    
    
    for x in basis:
        if sum(x)!=N:
            l0=len(basis_new)
            L=list(itertools.product(range(-ss2,ss2),repeat=l0))
            for y in L:
                s=vector(x)+sum(y[i]*vector(basis_new[i]) for i in range(0,l0))
                if sum(s)==N:
                    basis_new+=[list(s)]
                    break
    
    

    NN=[N for i in range(0,d+1)]
    coord_old=diagonal_matrix(NN)


    coord_new=(coord_old*matrix(basis_new).inverse()).rows()
    coord=coord_new
    out=[list(z)[1:] for z in coord]
    if h_star_code(G,N)!=h_star(out):
        return 'ERROR'
    return out

def lN_sc(l,s):
        M=range(-s,s+1)
        M=itertools.product(M, repeat=l)
        M=list(filter(lambda x: (sum(x)<=s)&(sum(x)>-s),M))
        return M

In [14]:
simplex_from_code([[1,1,0],[0,1,1]],2,1,3)

[[0, -1], [2, -1], [0, 1]]

In [64]:
thin_codes(pool,4,2,4)

([((0, 0, 0, 1, 1), (1, 1, 1, 0, 1)),
  ((0, 0, 0, 1, 1), (1, 1, 1, 0, 1), (0, 0, 1, 1, 0)),
  ((0, 0, 0, 1, 1), (1, 1, 1, 0, 1), (0, 0, 1, 1, 0), (0, 1, 0, 1, 0))],
 [(
         [0 0 0 1 1]                                             
0, 'G=', [1 1 1 0 1], 'h*=', 2*t^2 + t + 1, 'W=', 2*X^4 + X^2 + 1,

'f_h', 2*t^2 + t + 1, 'f_W', 2*Y^4 + Y^2 + 1, '   '
),
  (
         [0 0 0 1 1]                                                 
         [1 1 1 0 1]                                                 
1, 'G=', [0 0 1 1 0], 'h*=', 3*t^2 + 4*t + 1, 'W=', 3*X^4 + 4*X^2 + 1,

'f_h', (t + 1) * (3*t + 1), 'f_W', (Y^2 + 1) * (3*Y^2 + 1),

'FREE JOIN likely'
),
  (
         [0 0 0 1 1]                               
         [1 1 1 0 1]                               
         [0 0 1 1 0]                               
2, 'G=', [0 1 0 1 0], 'h*=', 5*t^2 + 10*t + 1, 'W=',

5*X^4 + 10*X^2 + 1, 'f_h', 5*t^2 + 10*t + 1, 'f_W', 5*Y^4 + 10*Y^2 + 1,

'   '
)])

In [68]:
thin_codes(pool,4,3,4)

([((0, 0, 2, 2, 2), (1, 2, 0, 1, 2))],
 [(
         [0 0 2 2 2]                                               
0, 'G=', [1 2 0 1 2], 'h*=', 7*t^2 + t + 1, 'W=', 6*X^4 + 2*X^3 + 1,

'f_h', 7*t^2 + t + 1, 'f_W', 6*Y^4 + 2*Y^3 + 1, '   '
)])

In [69]:
thin_codes(pool,4,4,4)

([((0, 1, 2, 2, 3), (1, 0, 2, 3, 2)),
  ((0, 1, 2, 2, 3), (2, 1, 0, 2, 3)),
  ((0, 2, 2, 2, 2), (3, 0, 1, 2, 2)),
  ((0, 2, 2, 2, 2), (3, 0, 1, 2, 2), (1, 0, 1, 0, 2)),
  ((0, 1, 2, 2, 3), (2, 1, 0, 2, 3), (0, 1, 0, 0, 3))],
 [(
         [0 1 2 2 3]                                     
0, 'G=', [1 0 2 3 2], 'h*=', t^3 + 11*t^2 + 3*t + 1, 'W=',

13*X^4 + 2*X^2 + 1, 'f_h', t^3 + 11*t^2 + 3*t + 1, 'f_W',

13*Y^4 + 2*Y^2 + 1, '   '
),
  (
         [0 1 2 2 3]                                                 
1, 'G=', [2 1 0 2 3], 'h*=', 5*t^2 + 2*t + 1, 'W=', 5*X^4 + 2*X^2 + 1,

'f_h', 5*t^2 + 2*t + 1, 'f_W', 5*Y^4 + 2*Y^2 + 1, '   '
),
  (
         [0 2 2 2 2]                              
2, 'G=', [3 0 1 2 2], 'h*=', 5*t^2 + 2*t + 1, 'W=',

4*X^4 + 2*X^3 + X^2 + 1, 'f_h', 5*t^2 + 2*t + 1, 'f_W',

4*Y^4 + 2*Y^3 + Y^2 + 1, '   '
),
  (
         [0 2 2 2 2]                              
         [3 0 1 2 2]                              
3, 'G=', [1 0 1 0 2], 'h*=', 9*t^2 + 6*t + 1, 'W=',

7*

In [72]:
thin_codes(pool,4,6,4)

([((0, 1, 3, 3, 5), (3, 1, 0, 3, 5)),
  ((0, 2, 3, 3, 4), (3, 0, 0, 3, 0)),
  ((0, 2, 3, 3, 4), (3, 0, 0, 3, 0), (0, 5, 0, 0, 1))],
 [(
         [0 1 3 3 5]                                                 
0, 'G=', [3 1 0 3 5], 'h*=', 8*t^2 + 3*t + 1, 'W=', 8*X^4 + 3*X^2 + 1,

'f_h', 8*t^2 + 3*t + 1, 'f_W', 8*Y^4 + 3*Y^2 + 1, '   '
),
  (
         [0 2 3 3 4]                                                 
1, 'G=', [3 0 0 3 0], 'h*=', 6*t^2 + 5*t + 1, 'W=', 6*X^4 + 5*X^2 + 1,

'f_h', (2*t + 1) * (3*t + 1), 'f_W', (2*Y^2 + 1) * (3*Y^2 + 1),

'FREE JOIN likely'
),
  (
         [0 2 3 3 4]                               
         [3 0 0 3 0]                               
2, 'G=', [0 5 0 0 1], 'h*=', 15*t^2 + 8*t + 1, 'W=',

15*X^4 + 8*X^2 + 1, 'f_h', (3*t + 1) * (5*t + 1), 'f_W',

(3*Y^2 + 1) * (5*Y^2 + 1), 'FREE JOIN likely'
)])

In [71]:
thin_codes(pool,4,8,4)

([((0, 3, 4, 4, 5), (4, 5, 0, 4, 3)),
  ((0, 2, 3, 4, 7), (4, 4, 4, 4, 0)),
  ((0, 3, 4, 4, 5), (4, 5, 0, 4, 3), (0, 5, 0, 0, 3))],
 [(
         [0 3 4 4 5]                               
0, 'G=', [4 5 0 4 3], 'h*=', 11*t^2 + 4*t + 1, 'W=',

11*X^4 + 4*X^2 + 1, 'f_h', 11*t^2 + 4*t + 1, 'f_W', 11*Y^4 + 4*Y^2 + 1,

'   '
),
  (
         [0 2 3 4 7]                                     
1, 'G=', [4 4 4 4 0], 'h*=', t^3 + 11*t^2 + 3*t + 1, 'W=',

12*X^4 + 2*X^3 + X^2 + 1, 'f_h', t^3 + 11*t^2 + 3*t + 1, 'f_W',

12*Y^4 + 2*Y^3 + Y^2 + 1, '   '
),
  (
         [0 3 4 4 5]                                
         [4 5 0 4 3]                                
2, 'G=', [0 5 0 0 3], 'h*=', 21*t^2 + 10*t + 1, 'W=',

21*X^4 + 10*X^2 + 1, 'f_h', (3*t + 1) * (7*t + 1), 'f_W',

(3*Y^2 + 1) * (7*Y^2 + 1), 'FREE JOIN likely'
)])

In [2]:
thin_codes_experimental(pool,4,4,3,0,0)

number of 2-dim  265 0.2605409622192383
checking weights  2
done 2 0.3144690990447998
equiv check 2
equiv check 2 done
len  3 is 509
number of 3-dim  509 0.3620872497558594
checking weights  3
done
0.7026190757751465
equiv check  3
equiv check done  3


([((0, 1, 2, 2, 3), (2, 3, 0, 0, 3)),
  ((0, 1, 2, 2, 3), (2, 3, 0, 2, 1)),
  ((0, 1, 2, 2, 3), (1, 3, 0, 1, 3)),
  ((0, 1, 2, 2, 3), (2, 3, 0, 2, 1), (0, 2, 0, 2, 0)),
  ((0, 1, 2, 2, 3), (2, 3, 0, 2, 1), (0, 3, 0, 0, 1))],
 [(
         [0 1 2 2 3]                              
0, 'G=', [2 3 0 0 3], 'h*=', 5*t^2 + 2*t + 1, 'W=',

4*X^4 + 2*X^3 + X^2 + 1, 'f_h', 5*t^2 + 2*t + 1, 'f_W',

4*Y^4 + 2*Y^3 + Y^2 + 1, ' '
),
  (
         [0 1 2 2 3]                                                 
1, 'G=', [2 3 0 2 1], 'h*=', 5*t^2 + 2*t + 1, 'W=', 5*X^4 + 2*X^2 + 1,

'f_h', 5*t^2 + 2*t + 1, 'f_W', 5*Y^4 + 2*Y^2 + 1, ' '
),
  (
         [0 1 2 2 3]                                     
2, 'G=', [1 3 0 1 3], 'h*=', t^3 + 11*t^2 + 3*t + 1, 'W=',

13*X^4 + 2*X^2 + 1, 'f_h', t^3 + 11*t^2 + 3*t + 1, 'f_W',

13*Y^4 + 2*Y^2 + 1, ' '
),
  (
         [0 1 2 2 3]                              
         [2 3 0 2 1]                              
3, 'G=', [0 2 0 2 0], 'h*=', 9*t^2 + 6*t + 1, 'W=',

7*X^4 + 